# aDEG Direction Exploration
Code conceptualized and written by Monica E. Mesecar with aesthetic support by Perplexity AI.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
from matplotlib import rcParams

In [ ]:
!pwd

In [ ]:
p1_adeg=pd.read_csv("P1_aging.glmmtmb_age_diffs_fdr_cleaned.csv")

In [ ]:
display(p1_adeg)

In [ ]:
#Create lists of subsets of interest
p1_type_list=p1_adeg.type.unique().tolist()
print(p1_type_list)

In [ ]:
#Subset only 'region_broad_celltype'
p1_region_broad_df = p1_adeg.loc[p1_adeg['type'] == 'region_broad_celltype']

In [ ]:
#Show df
p1_region_broad_df 

In [ ]:
p1_region_broad_df  = p1_region_broad_df [p1_region_broad_df ['celltype'] != 'Indeterminate']

In [ ]:
unique_count = p1_region_broad_df['feature'].nunique()
print(unique_count)

In [ ]:
custom_celltype_order = ['InN', 'ExN', 'SPN', 'OPC', 'Oligodendrocyte', 'Astrocyte', 'Microglia', 'Ependymal', 'Endothelial', 'Mural']  # Replace with your desired order

In [ ]:
#All aDEGs including shared 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = p1_region_broad_df

cell_types = cell_types = custom_celltype_order
all_regions = sorted(df['region'].unique())
cols = 3
rows = int(np.ceil(len(cell_types) / cols))
soft_red = '#f4a582'
soft_blue = '#92c5de'

fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
axes = axes.flatten()

for i, cell_type in enumerate(cell_types):
    ax = axes[i]
    df_ct = df[df['celltype'] == cell_type]
    counts = df_ct.groupby(['region', 'direction']).size().unstack(fill_value=0)
    counts = counts.reindex(index=all_regions, columns=[-1, 1], fill_value=0)

    regions = counts.index.tolist()
    down_counts = counts[-1].values
    up_counts = counts[1].values
    totals = down_counts + up_counts

    bar_width = 0.6
    bars_down = ax.bar(regions, down_counts, color=soft_blue, width=bar_width, label='Down')
    bars_up = ax.bar(regions, up_counts, bottom=down_counts, color=soft_red, width=bar_width, label='Up')

    for j, (down, up) in enumerate(zip(down_counts, up_counts)):
        total = down + up
        if total > 0:
            ax.text(j, down/2, f'{down/total:.0%}', ha='center', va='center', color='black', fontsize=10)
            ax.text(j, down + up/2, f'{up/total:.0%}', ha='center', va='center', color='black', fontsize=10)

    ax.set_xticks(range(len(all_regions)))
    ax.set_xticklabels(all_regions, rotation=45, ha='right')
    ax.set_title(f'{cell_type}', fontsize=14, pad=12)
    ax.set_xlabel('Region', fontsize=12)
    ax.set_ylabel('Gene Count', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.set_ylim(0, max(totals)*1.2 if totals.any() else 5)

for j in range(len(cell_types), len(axes)):
    fig.delaxes(axes[j])

# Add a single legend for the whole figure
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=2, fontsize=12, frameon=False)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.savefig('ALL_aDEG_dir_plots.pdf', transparent=True)
plt.show()